# Research Project

## Import required libraries

In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install rouge_score bert_score sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import list_datasets, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, get_polynomial_decay_schedule_with_warmup
from torch.nn import functional as F
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from itertools import chain
import torch
import math
import numpy as np
import random
import datasets

#For networking purposes
import os, sys
os.environ['CURL_CA_BUNDLE'] = ''

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [3]:
selected_model = 'dialoGPT' #'gpt2' 

## Tokenizer

In [4]:
if selected_model == 'dialoGPT':
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
elif selected_model == 'gpt2':
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
else:
    print('No tokenizer')

In [5]:
space = 'Ġ'
pre_quote = '’'
end_marks = ['.', ',', '?', '!', '...']
quotes = ['"', '\'']
abbreviations = ['s', 'd', 't', 'm', 're', 'll', 've', 'S', 'D', 'T', 'M', 'Re', 'Ll', 'Ve']

def process_token_list(token_list):
    token_list[0] = token_list[0].capitalize()
    
    quote_count = 0
    for i, token in enumerate(token_list):
        if space in token:
            if token[1:] in end_marks or token[1:] in abbreviations:
                token_list[i] = token[1:]
                
            if token[1:] == quotes[1]:
                if i<len(token_list)-1:
                    if token_list[i+1] in abbreviations or (token_list[i+1][0] == space and token_list[i+1][1:] in abbreviations):
                        token_list[i] = token[1:]
                        
        if token[0] == space and token[1:] in quotes:
            if quote_count % 2 == 1:
                token_list[i] = token[1:]
                quote_count = 0
            else:
                if i<len(token_list)-1 and token_list[i+1][0] == space:
                    token_list[i+1] = token_list[i+1][1:]
                quote_count += 1
                
        if token in end_marks or token[1:] in end_marks:
            if i<len(token_list)-1:
                if token_list[i+1][0] != space:
                    token_list[i+1] = space + token_list[i+1].capitalize()
                else:
                    token_list[i+1] = space + token_list[i+1][1:].capitalize()
                
    new_token_list = [token for token in token_list if token != space and len(token)>0]
    if new_token_list[-1] not in end_marks:
        new_token_list.append(end_marks[0])
        
    return new_token_list

## Load dataset

In [6]:
dataset = load_dataset('daily_dialog')
train_dialogues = dataset['train']['dialog']
valid_dialogues = dataset['validation']['dialog']
test_dialogues = dataset['test']['dialog']

  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
def load_daily(dataset, tokenizer):
        
    for i, dialogue in enumerate(tqdm(dataset)):
        new_dialogue = []
        for utter in dialogue:
            token_list = tokenizer.tokenize(utter.strip().replace(pre_quote, quotes[1]))
            token_list = process_token_list(token_list)
            text = tokenizer.convert_tokens_to_string(token_list)
            new_dialogue.append(text)
            
        dataset[i] = new_dialogue
    
    utter_num = 0

    for dialogue in dataset:
        utter_num += len(dialogue)
    
    return dataset, utter_num

In [8]:
train_dialogues, num_train = load_daily(train_dialogues, tokenizer)
valid_dialogues, num_valid = load_daily(valid_dialogues, tokenizer)
test_dialogues, num_test = load_daily(test_dialogues, tokenizer)

100%|██████████| 1000/1000 [00:00<00:00, 1194.91it/s]


In [9]:
print(f"The number of train dialogues: {len(train_dialogues)}")
print(f"The number of valid dialogues: {len(valid_dialogues)}")    
print(f"The number of test dialogues: {len(test_dialogues)}")    

print(f"The number of train utterances: {num_train}")    
print(f"The number of valid utterances: {num_valid}")
print(f"The number of test utterances: {num_test}")



The number of train dialogues: 11118
The number of valid dialogues: 1000
The number of test dialogues: 1000
The number of train utterances: 87170
The number of valid utterances: 8069
The number of test utterances: 7740


In [10]:
# Extrac ids (input_ids, token_ids) from processed text
def extract_ids(dialogues):

    ids = []
    for dialogue in tqdm(dialogues):
            dialogue_ids = []
            for utter in dialogue:
                tokens = tokenizer.tokenize(utter)
                token_ids = tokenizer.convert_tokens_to_ids(tokens)
                dialogue_ids.append(token_ids)
            ids.append(dialogue_ids)
            
    assert len(ids) == len(dialogues)

    return ids  

In [11]:
train_ids = extract_ids(train_dialogues)
valid_ids = extract_ids(valid_dialogues) 
# test_ids = extract_ids(test_dialogues) 

100%|██████████| 1000/1000 [00:00<00:00, 1446.59it/s]


In [12]:
#Parameters

sp1_token = '<sp1>'
sp2_token = '<sp2>'
bos_token = '<bos>'
max_turns = 5
max_len = 1024
seed = 0
gpu = 0

#Tokeniser
special_tokens = {'bos_token': bos_token,
                'additional_special_tokens': [sp1_token, sp2_token]}

eos_token = tokenizer.eos_token
num_new_tokens = tokenizer.add_special_tokens(special_tokens)

vocab = tokenizer.get_vocab()
vocab_size = len(vocab)
bos_id = vocab[bos_token]
eos_id = vocab[eos_token]
sp1_id = vocab[sp1_token]
sp2_id = vocab[sp2_token]

lr = 2e-5
batch_size = 8
num_workers = 0
num_epochs = 8
warmup_ratio = 0.1
last_epoch = 0
end_command = 'Quit!'
top_p = 0.8


In [13]:
!mkdir 'saved_models'
ckpt_dir = 'saved_models'

In [14]:
class CustomDataset(Dataset):
    def __init__(self, dials):

        self.input_ids = []  # (N, L)
        self.token_type_ids = []  # (N, L)
        self.labels = []  # (N, L)
            
        for dial in tqdm(dials):
            hists = []
            for u, utter in enumerate(dial):
                if u % 2 == 0:
                    hists.append([sp1_id] + utter)
                else:
                    hists.append([sp2_id] + utter)

            # print(hists) 
            # print() 
            for h in range(len(hists)):
                if hists[h][0] == sp2_id:
                    # print(hists[h])
                    start = max(0, h - max_turns+1)
                    # print('start: ', start, ' to: ', h)
                    for s in range(start, h):
                        contexts = hists[s:h+1]
                        # print('Context: ', contexts)
                        input_ids = [bos_id] + list(chain.from_iterable(contexts)) + [eos_id]
                        if len(input_ids) <= max_len:
                            start_sp_id, next_sp_id = contexts[0][0], contexts[1][0]
                            token_type_ids = [[start_sp_id] * len(ctx) if c % 2 == 0 else [next_sp_id] * len(ctx) for c, ctx in enumerate(contexts)]
                            # print('token_type_ids 1: ', token_type_ids)
                            # print('LEN 1: ', len(token_type_ids))
                            # print('len input_ids', len(input_ids))
                            assert token_type_ids[-1][0] == sp2_id
                            token_type_ids = [start_sp_id] + list(chain.from_iterable(token_type_ids)) + [sp2_id]
                            # print('token_type_ids 2: ', token_type_ids)
                            assert len(input_ids) == len(token_type_ids)
                            
                            labels = [[-100] * len(ctx) if c < len(contexts)-1 else [-100] + ctx[1:] for c, ctx in enumerate(contexts)]
                            # print('labels 1: ', labels)
                            assert labels[-1][1:] == contexts[-1][1:]
                            labels = [-100] + list(chain.from_iterable(labels)) + [eos_id]
                            # print('labels 2: ', labels)
                            assert len(input_ids) == len(labels)
                            
                            self.input_ids.append(input_ids)
                            self.token_type_ids.append(token_type_ids)
                            self.labels.append(labels)
                            
                            break
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.token_type_ids[idx], self.labels[idx]
    
    
class PadCollate():
    def __init__(self, eos_id):
        self.eos_id = eos_id
        
    def pad_collate(self, batch):
        input_ids, token_type_ids, labels =[], [], []
        for idx, seqs in enumerate(batch):
            input_ids.append(torch.LongTensor(seqs[0]))
            token_type_ids.append(torch.LongTensor(seqs[1]))
            labels.append(torch.LongTensor(seqs[2]))
            
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=self.eos_id)
        token_type_ids = torch.nn.utils.rnn.pad_sequence(token_type_ids, batch_first=True, padding_value=self.eos_id)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
    
        return input_ids, token_type_ids, labels

In [ ]:
print(eos_id, bos_id, sp1_id, sp2_id)

50256 50257 50258 50259


In [ ]:
print(train_dialogues[0])

['Say, Jim, How about going for a few beers after dinner?', 'You know that is tempting but is really not good for our fitness.', 'What do you mean? It will help us to relax.', "Do you really think so? I don't. It will just make us fat and act silly. Remember last time?", "I guess you are right. But what shall we do? I don't feel like sitting at home.", 'I suggest a walk over to the gym where we can play singsong and meet some of our friends.', "That's a good idea. I hear Mary and Sally often go there to play pingpong. Perhaps we can make a foursome with them.", 'Sounds great to me! If they are willing, We could ask them to go dancing with us. That is excellent exercise and fun, Too.', "Good. Let's go now.", 'All right.']


In [ ]:
#Debugging purposes
debug_dialog = CustomDataset([train_ids[0]])

100%|██████████| 1/1 [00:00<00:00, 4194.30it/s]


In [15]:
def fix_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)

In [16]:
#Load Model

if torch.cuda.is_available():
    device = torch.device(f"cuda:{gpu}")
    print('Using GPU')
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU


In [18]:
print("Loading the model: ", selected_model)
fix_seed(seed)

if selected_model == 'dialoGPT':
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small").to(device)
elif selected_model == 'gpt2':
    model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
else:
    print('No model')

model.resize_token_embeddings(vocab_size)
max_len = min(max_len, model.config.n_ctx)

Loading the model:  dialoGPT


In [33]:
#Load from checkpoint
ckpt = torch.load("/content/saved_models/best_ckpt_epoch=4_valid_loss=2.5025.ckpt", map_location=device)
model.load_state_dict(ckpt['model_state_dict'])

<All keys matched successfully>

In [19]:
# Load optimizer
print("Loading the optimizer...")
optim = torch.optim.AdamW(model.parameters(), lr=lr)

Loading the optimizer...


In [20]:
# Load train & valid dataset
print("Loading train & valid data...")

train_set = CustomDataset(train_ids)
valid_set = CustomDataset(valid_ids)
# test_set = CustomDataset(test_ids)

ppd = PadCollate(eos_id=eos_id)

train_loader = DataLoader(train_set, 
                            collate_fn=ppd.pad_collate, 
                            shuffle=True, 
                            batch_size=batch_size, 
                            num_workers=num_workers, 
                            pin_memory=True)

valid_loader = DataLoader(valid_set, 
                            collate_fn=ppd.pad_collate,
                            batch_size=batch_size, 
                            num_workers=num_workers, 
                            pin_memory=True)

# test_loader = DataLoader(test_set, 
#                             collate_fn=ppd.pad_collate,
#                             batch_size=batch_size, 
#                             num_workers=num_workers, 
#                             pin_memory=True)
    
# Calculate total training steps
num_batches = len(train_loader)
total_train_steps = num_epochs * num_batches
warmup_steps = int(warmup_ratio * total_train_steps)

sched = get_polynomial_decay_schedule_with_warmup(
    optim,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_train_steps,
    power=2
)

writer = SummaryWriter()

Loading train & valid data...


100%|██████████| 1000/1000 [00:00<00:00, 21125.31it/s]


In [21]:
def validation():

    print("Validation processing...")
    model.eval()
            
    valid_losses = []
    valid_ppls = []
    with torch.no_grad():
        for i, batch in enumerate(tqdm(valid_loader)):
            input_ids, token_type_ids, labels = batch
            input_ids, token_type_ids, labels = \
                input_ids.to(device), token_type_ids.to(device), labels.to(device)
            
            outputs = model(
                input_ids=input_ids,
                token_type_ids = token_type_ids,
                labels = labels
            )
            
            loss, logits = outputs[0], outputs[1]
            
            valid_losses.append(loss.detach())
            ppl = torch.exp(loss.detach())
            valid_ppls.append(ppl)
        
        valid_losses = [loss.item() for loss in valid_losses]
        valid_ppls = [ppl.item() if not math.isinf(ppl.item()) else 1e+8 for ppl in valid_ppls]
        valid_loss = np.mean(valid_losses)
        valid_ppl = np.mean(valid_ppls)
        
        if math.isnan(valid_ppl):
            valid_ppl = 1e+8
            
    return valid_loss, valid_ppl

In [22]:
def train():
    
    fix_seed(seed)  # Fix seed before training
    print("Training starts.")

    best_loss = sys.float_info.max
    last_epoch= 0
    
    start_epoch = last_epoch +1
    for epoch in range(start_epoch, start_epoch+num_epochs):
        model.train()
        
        print(f"#"*50 + f"Epoch: {epoch}" + "#"*50)
        train_losses = []
        train_ppls = []
        for i, batch in enumerate(tqdm(train_loader)):
            input_ids, token_type_ids, labels = batch
            input_ids, token_type_ids, labels = \
                input_ids.to(device), token_type_ids.to(device), labels.to(device)
            
            outputs = model(
                input_ids=input_ids,
                token_type_ids = token_type_ids,
                labels = labels
            )
            
            loss, logits = outputs[0], outputs[1]
            
            optim.zero_grad()
            loss.backward()
            optim.step()
            sched.step()
            
            train_losses.append(loss.detach())
            ppl = torch.exp(loss.detach())
            train_ppls.append(ppl)
        
        train_losses = [loss.item() for loss in train_losses]
        train_ppls = [ppl.item() if not math.isinf(ppl.item()) else 1e+8 for ppl in train_ppls]
        train_loss = np.mean(train_losses)
        train_ppl = np.mean(train_ppls)
        print(f"Train loss: {train_loss} || Train perplexity: {train_ppl}")
        
        writer.add_scalar("Loss/train", train_loss, epoch)
        writer.add_scalar("PPL/train", train_ppl, epoch)
        
        last_epoch += 1
        
        valid_loss, valid_ppl = validation()
            
        if valid_loss < best_loss:
            best_loss = valid_loss
            state_dict = {
                'model_state_dict': model.state_dict(),
                'optim_state_dict': optim.state_dict(),
                'sched_state_dict': sched.state_dict(),
                'loss': best_loss,
                'epoch': last_epoch
            }
            
            torch.save(state_dict, f"{ckpt_dir}/best_ckpt_epoch={epoch}_valid_loss={round(best_loss, 4)}.ckpt")
            print("*"*10 + "Current best checkpoint is saved." + "*"*10)
            print(f"{ckpt_dir}/best_ckpt_epoch={epoch}_valid_loss={round(best_loss, 4)}.ckpt")
            
        print(f"Best valid loss: {best_loss}")
        print(f"Valid loss: {valid_loss} || Valid perplexity: {valid_ppl}")
        
        writer.add_scalar("Loss/valid", valid_loss, epoch)
        writer.add_scalar("PPL/valid", valid_ppl, epoch)
        
        writer.add_scalars("Losses", {
            'train': train_loss, 
            'valid': valid_loss,
        }, epoch)
        writer.add_scalars("PPLs", {
            'train': train_ppl,
            'valid': valid_ppl,
        }, epoch)
            
    print("Training finished!")

In [ ]:
train()

Training starts.
##################################################Epoch: 1##################################################


100%|██████████| 5205/5205 [25:30<00:00,  3.40it/s]


Train loss: 3.622699253176635 || Train perplexity: 149.9571120286881
Validation processing...


100%|██████████| 482/482 [00:37<00:00, 12.99it/s]


**********Current best checkpoint is saved.**********
saved_models/best_ckpt_epoch=1_valid_loss=2.6851.ckpt
Best valid loss: 2.6850774725937745
Valid loss: 2.6850774725937745 || Valid perplexity: 15.70372484432711
##################################################Epoch: 2##################################################


 85%|████████▌ | 4439/5205 [21:49<03:31,  3.62it/s]

In [ ]:
window = 5

def infer():
    model.eval()
    fix_seed(seed)

    generated_responses = []
    actual_responses = []

    with torch.no_grad():
        
        for i, batch in enumerate(tqdm(test_dialogues)):

            # print()
            # print(batch)
            input_hists = []
            context = []
            for j in range(0, len(batch)): #Note: dialogues < window won't be processed

                #Set speaker 1 or speaker 2
                sp_id = sp1_id if j % 2 == 0 else sp2_id

                #Get utterance
                utter = batch[j]
            
                input_ids = [sp_id] + tokenizer.encode(utter)
                input_hists.append(input_ids)

                #Context just for debugging
                context.append(utter)
                
                if len(input_hists) < window:
                    continue
                elif len(input_hists) > window:
                    input_hists = input_hists[-window:]
                    context = context[-window:]
                
                # Debugging
                if i % 200 == 0:
                    print()
                    print('Context:')
                    for c in context:
                        print(c)
                
                # print('input_hists: ', input_hists)
                # print('len input_hists: ', len(input_hists))

                start_sp_id = input_hists[0][0]
                next_sp_id = sp1_id if start_sp_id == sp2_id else sp2_id
                assert start_sp_id != next_sp_id

                input_ids = [bos_id] + list(chain.from_iterable(input_hists)) + [next_sp_id] #Because window is 5, so 6th utter is = sp2
                # print('input_hists with bos: ', input_ids)
                
                token_type_ids = [[start_sp_id] * len(hist) if h % 2 == 0 else [next_sp_id] * len(hist) for h, hist in enumerate(input_hists)]
                assert len(token_type_ids) == len(input_hists)
                token_type_ids = [start_sp_id] + list(chain.from_iterable(token_type_ids)) + [next_sp_id]
                # print('token_type_ids: ', token_type_ids)
                assert len(input_ids) == len(token_type_ids)
                input_len = len(input_ids)
            
                input_ids = torch.LongTensor(input_ids).unsqueeze(0).to(device)
                token_type_ids = torch.LongTensor(token_type_ids).unsqueeze(0).to(device)
                
                # next_sp_id = sp2_id if input_hists[-1][0] == sp1_id else sp1_id

                output_ids = nucleus_sampling(input_ids, token_type_ids, input_len, next_sp_id)                
            # output_ids = self.model.generate(
            #     input_ids=input_ids, token_type_ids=token_type_ids, pad_token_id=self.args.eos_id,
            #     do_sample=True, top_p=self.args.top_p, max_length=self.args.max_len,
            #     output_hidden_states=True, output_scores=True, return_dict_in_generate=True,
            # ).sequences
            # output_ids = output_ids[0].tolist()[input_len:]
                res = tokenizer.decode(output_ids, skip_special_tokens=True)

                if j == len(batch) - 1:  # No subsequent sentence
                    continue
                
                actual_res = batch[j+1]
                
                if i % 200 == 0:
                    print()
                    print(f"Bot response: {res}")
                    print(f"Actual response: {actual_res}")

                generated_responses.append(res)
                actual_responses.append(actual_res)

                # input_hists.append([next_sp_id] + tokenizer.encode(actual_res))
                # context.append(actual_res)

                # print('final input_hists: ', input_hists)    

               
    
    return generated_responses, actual_responses

            
                
def nucleus_sampling(input_ids, token_type_ids, input_len, next_sp_id):
    output_ids = []
    for pos in range(input_len, max_len):
        output = model(input_ids=input_ids, token_type_ids=token_type_ids)[0][:, pos-1]  # (1, V)
        output = F.softmax(output, dim=-1)  # (1, V)
        
        sorted_probs, sorted_idxs = torch.sort(output, descending=True)
        cumsum_probs = torch.cumsum(sorted_probs, dim=-1)  # (1, V)
        idx_remove = cumsum_probs > top_p
        idx_remove[:, 1:] = idx_remove[:, :-1].clone()
        idx_remove[:, 0] = False
        sorted_probs[idx_remove] = 0.0
        sorted_probs /= torch.sum(sorted_probs, dim=-1, keepdim=True)  # (1, V)
        
        probs = torch.zeros(output.shape, device=device).scatter_(-1, sorted_idxs, sorted_probs)  # (1, V)
        idx = torch.multinomial(probs, 1)  # (1, 1)
        
        idx_item = idx.squeeze(-1).squeeze(-1).item()
        output_ids.append(idx_item)
        
        if idx_item == eos_id:
            break
            
        input_ids = torch.cat((input_ids, idx), dim=-1)
        next_type_id = torch.LongTensor([[next_sp_id]]).to(device)
        token_type_ids = torch.cat((token_type_ids, next_type_id), dim=-1)
        assert input_ids.shape == token_type_ids.shape
        
    return output_ids

In [ ]:
generated_responses, actual_responses = infer()

In [ ]:
assert len(generated_responses) == len(actual_responses)
print(len(generated_responses))
print(len(actual_responses))

### Store responses

In [ ]:
import pickle
from google.colab import files

file_generated = "" + selected_model + "_batch8_generated_responses"
file_actual = "" + selected_model + "_batch8_actual_responses"

with open(file_generated, "wb") as fp:
    pickle.dump(generated_responses, fp)

with open(file_actual, "wb") as fp:
    pickle.dump(actual_responses, fp)

files.download(file_generated) 
files.download(file_actual) 

In [43]:
#Loading example
# with open("gpt2_batch8_generated_responses", "rb") as fp:   # Unpickling
#     dummy = pickle.load(fp)

### Compute metrics

In [ ]:
import evaluate

sacrebleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
chrf = evaluate.load("chrf")

In [ ]:
actual_responses = [[res] for res in actual_responses] #Refs must be in a list of list of str

print(generated_responses[:5])
print(actual_responses[:5])

In [ ]:
bleu_score = sacrebleu.compute(predictions=generated_responses, references=actual_responses)

rouge_score = rouge.compute(predictions=generated_responses, references=actual_responses)

bert_score = bertscore.compute(predictions=generated_responses, references=actual_responses, lang='en')
precision = bert_score['precision']
recall = bert_score['recall']
f1 = bert_score['f1']
avg_precision_bert = sum(precision) / len(precision)
avg_recall_bert = sum(recall) / len(recall)
avg_f1_bert = sum(f1) / len(f1)

chrf_score = chrf.compute(predictions=generated_responses, references=actual_responses)

In [ ]:
print('Bleu score: \n', bleu_score) #Range from 0 to 100
print('Rouge score: \n', rouge_score)
print('Bert score: \n', bert_score)
print('Avg precision Bert score: ', avg_precision_bert)
print('Avg recall Bert score: ', avg_recall_bert)
print('Avg f1 Bert score: ', avg_f1_bert)
print('chrf score: \n', chrf_score)


In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [57]:
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [["hello there general kenobi", "hello there !"],
                 ["foo bar foobar", "foo bar foobar"]]
sacrebleu = evaluate.load("sacrebleu")
results = sacrebleu.compute(predictions=predictions, 
                             references=references)
print(results)

results = rouge.compute(predictions=predictions, 
                             references=references)
print(results)

results = bertscore.compute(predictions=predictions, 
                             references=references, lang='eng')
print(results)

results = chrf.compute(predictions=predictions, 
                             references=references)
print(results)

{'score': 100.00000000000004, 'counts': [7, 5, 3, 1], 'totals': [7, 5, 3, 1], 'precisions': [100.0, 100.0, 100.0, 100.0], 'bp': 1.0, 'sys_len': 7, 'ref_len': 7}
{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


{'precision': [0.9999999403953552, 1.0], 'recall': [0.9999999403953552, 1.0], 'f1': [0.9999999403953552, 1.0], 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.30.0)'}
{'score': 100.0, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [48]:
# !cp '/content/saved_models/best_ckpt_epoch=4_valid_loss=2.5025.ckpt' .